# Honey Bee Classifier
## Streamlit Web App
  - In order to create a web app through Colab to implement the CNNs for subspecies and health classification, we must install and run packages allowing for a secure localtunnel connection to be made.

  - This notebook takes advantage of Google Drive. After forking and cloning the repo from Github, upload the files to Google Drive and change your file paths accordingly.

#### Mount Google Drive
 - Authentication through a pop-up window is required. Mounting Drive allows for importing models and data uploaded to Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Create Directories
 - Directories  are created to house the user-uploaded image and the RGB converted image. Directories are deleted upon disconnecting the runtime.

In [2]:
!mkdir uploaded_images
!mkdir rgb_images

mkdir: cannot create directory ‘uploaded_images’: File exists
mkdir: cannot create directory ‘rgb_images’: File exists


#### Install Streamlit

In [3]:
!pip install -q streamlit

#### Install Localtunnel

In [4]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.529s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



### Streamlit App
 - The entire app is written to an app.py file upon running this notebook. The file is re-written upon disconnecting and reconnecting the runtime.

In [5]:
%%writefile app.py

# Imports
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import tensorflow as tf
from tensorflow import keras
import imageio
import skimage
import skimage.io
import skimage.transform
from PIL import Image

# Global Variables
IMAGE_PATH = '/content/drive/MyDrive/Capstone/data/bee_imgs/'
TEST_IMAGE_PATH = '/content/drive/MyDrive/Capstone/data/test_bee_img/'
TEMP_PATH = '/content/rgbimg/'
MODEL_SUB_PATH = '/content/drive/MyDrive/Capstone/models/model_lr_subspecies_bal/'
MODEL_HEALTH_PATH = '/content/drive/MyDrive/Capstone/models/model_lr_subspecies_bal/'
IMAGE_WIDTH = 100
IMAGE_HEIGHT = 100
IMAGE_CHANNELS = 3
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
CONV_2D_DIM_1 = 16
CONV_2D_DIM_2 = 16
CONV_2D_DIM_3 = 32
CONV_2D_DIM_4 = 64
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
BATCH_SIZE = 32
PATIENCE = 5
NO_EPOCHS_1 = 5
NO_EPOCHS_2 = 10
NO_EPOCHS_3 = 50


# Predicted Class Labels
pred_species_labels = ['Unknown', '1 Mixed local stock 2', 'Carniolan honey bee',
       'Italian honey bee', 'Russian honey bee', 'VSH Italian honey bee',
       'Western honey bee']

pred_health_labels = ["Varroa", "Small Hive Beetles", "Ant Problems", 
"Few Varroa", "Hive Beetles, Healthy" , "Hive Being Robbed", "Missing Queen"]


# Loading Keras Models
# Change the filepath to reflect your uploaded path
model_subspecies_bal = keras.models.load_model(MODEL_SUB_PATH)
model_health_bal = keras.models.load_model(MODEL_HEALTH_PATH)

def read_rgb_image(file_path):
    image = skimage.io.imread(file_path)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), mode='reflect')
    return image[:,:,:IMAGE_CHANNELS]

def subsp_health_predictor(image_path):
    image = Image.open(image_path)
    rgb_image = image.convert('RGB')
    rgb_filename = image.filename.split('/')[-1]
    rgb_image.save('./rgb_images/'+ rgb_filename)
    rgb_image_path = './rgb_images/'+rgb_filename

    X = np.stack(read_rgb_image(rgb_image_path))
    X = X.reshape(1,X.shape[0],X.shape[1],X.shape[2])
    preds_subspecies_bal = model_subspecies_bal.predict(X)
    preds_health_bal = model_health_bal.predict(X)
    preds_species_bal_index = int(np.argmax(preds_subspecies_bal, axis = 1))
    p_s_b = pred_species_labels[preds_species_bal_index]

    preds_health_bal_index = int(np.argmax(preds_health_bal, axis = 1))
    p_h_b = pred_health_labels[preds_health_bal_index]

    return p_s_b, p_h_b  


def run():
    img1 = Image.open('/content/drive/MyDrive/Capstone/assets/splash_image.png')
    img1 = img1.resize((350,350))
    st.image(img1,use_column_width=False)
    st.title("Welcome to Adi's Honey Bee Classifier!")
    st.markdown('''<h4 style='text-align: left; color: #d73b5c;'>* This classifier uses a neural network trained on 5,100+ bee images annotated with subspecies, health condition, and other qualities, extracted from still time-lapse videos of bees."</h4>''',
                unsafe_allow_html=True)

    img_file = st.file_uploader("Choose an Image of Honey Bee!", type=["jpg", "png"])
    if img_file is not None:
        st.image(img_file,use_column_width=False)
        save_image_path = './uploaded_images/'+img_file.name
        with open(save_image_path, "wb") as f:
            f.write(img_file.getbuffer())



        if st.button("Predict"):
            species, health = subsp_health_predictor(save_image_path)
            st.success("Your bee's species is: " + species +' and the health status is ' + health+'.')
run()

Overwriting app.py


#### Run Streamlit in the background

In [6]:
!streamlit run /content/app.py &>/content/logs.txt &

#### Expose Port 8501
Click on the `url` to access the Streamlit App. A `log.txt` file will be created when run, and deleted upon disconnecting the runtime.

In [7]:
!npx localtunnel --port 8501

npx: installed 22 in 3.497s
your url is: https://floppy-baboons-hunt-34-75-94-7.loca.lt
^C
